In [49]:
import os,csv
import numpy as np
import tensorflow as tf
import keras
from mtcnn import MTCNN
from numpy import asarray
import matplotlib.pyplot as plt

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from PIL import Image
import hashlib
from flask import Flask, request, jsonify
import io
import threading
from skimage.filters import threshold_otsu


In [50]:
app = Flask(__name__)

def extract_face(pixels, required_size=(224, 224)):

    detector = MTCNN()
	
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
	
    face = pixels[y1:y2, x1:x2]
	
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    face_array = face_array.astype('float32')
    
    img_array = np.expand_dims(face_array, axis=0)
    img_array = preprocess_input(img_array, version=2)
    
    return img_array

def hash_features(features, output_dim=64):
    hasher = hashlib.sha256()
    hasher.update(features.tobytes())
    hash_digest = hasher.digest()
    # To get 64 bytes, concatenate two hash digests
    extended_hash = hash_digest + hashlib.sha256(hash_digest).digest()
    
    # Convert the extended hash to binary representation
    binary_hash = ''.join(format(byte, '08b') for byte in extended_hash)
    
    # Ensure the output_dim does not exceed the length of the binary hash
    binary_hash = binary_hash[:output_dim]
    
    # Convert the binary string to a numpy array of 0s and 1s
    hash_array = np.array(list(binary_hash), dtype=int)
    
    return hash_array


def extract_features(img):
    img_array = extract_face(img)

    # Charger le modèle SENet50 pré-entraîné
    model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

    # Extraire les features de l'image
    features = model.predict(img_array)
    # hashed_features = hash_features(features, output_dim=64)
    return features


@app.route('/extract_features', methods=['POST'])
def extract_features_route():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part in the request'}), 400

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        image = Image.open(file.stream)
        img=np.array(image)
        features = extract_features(img)
        return {"features": features.tolist()}
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [51]:
def run_app():
    app.run(debug=False, use_reloader=False)

# Run Flask app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
